# Lib

In [10]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import warnings

## ignore warning log in pandas
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import sys

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./../src")

from utils import utils_ml, parameters, metrics_evaluation, Pipeline_functions


from ipywidgets import interact

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
if sys.platform=='win32':
    !dir .\
        
elif sys.platform=='linux':
    !ls ./

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks

29/11/2022  02:35    <DIR>          .
23/11/2022  00:25    <DIR>          ..
10/09/2022  23:28    <DIR>          .ipynb_checkpoints
29/11/2022  02:35         1.008.994 1.EA.ipynb
25/11/2022  01:27            74.828 2.Feature_engineering.ipynb
29/11/2022  03:09            28.642 3.Stability_var.ipynb
05/12/2022  21:04           183.628 4.Modeling.ipynb
10/12/2022  16:10            65.343 5.Modelo_analisis_negocio.ipynb
04/12/2022  17:40           167.928 6.Pipeline.ipynb
25/11/2022  01:32           156.224 7.Pipeline_SPARK.ipynb
24/11/2022  23:44    <DIR>          catboost_info
               7 arquivo(s)      1.685.587 bytes
               4 pasta(s)   361.546.559.488 bytes dispon�veis


# Reading

In [12]:
## para saber en que directório uno se encuentra
#
if sys.platform=='win32':
    !echo %cd%
        
elif sys.platform=='linux':
    !pwd

c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks


In [13]:
!dir .\\..\data

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\data

04/12/2022  11:04    <DIR>          .
23/11/2022  00:25    <DIR>          ..
04/12/2022  11:04            24.658 cat_dataset.csv
10/08/2021  12:08        26.677.890 dados.csv
28/11/2022  00:10        14.104.434 dataset_structed.parquet
28/11/2022  00:05        27.547.133 EA.pbix
22/11/2022  22:34            39.018 iv_woe_by_week.parquet
09/11/2022  10:35            54.091 MySQL_localhost_connection.pbix
05/12/2022  19:33         3.891.056 test_dataset.parquet
05/12/2022  19:33         8.711.592 train_dataset.parquet
05/12/2022  19:33        14.493.356 train_test_dataset_tranformed.parquet
05/12/2022  19:33         2.094.822 validation_dataset.parquet
              10 arquivo(s)     97.638.050 bytes
               2 pasta(s)   361.546.559.488 bytes dispon�veis


In [14]:
df_origin = pd.read_parquet(f"{parameters.data_path}\\dataset_structed.parquet")

In [15]:
scores = [x for x in range(0,101)]

## 6. Cost Matrix and confusion Matrix

- Queremos encontrar o ponto de corte ideal para o modelo já treinado
apresentado na parte (Score, onde 0 (zero) é o risco mais baixo e 100 (cem) o
mais alto). Sabemos que MELI ganha 10% do valor de um pagamento
aprovado corretamente e a cada fraude aprovada perdemos 100% do valor do
pagamento.Defina o ponto de corte que maximize os lucros do MELI, de
forma que todos os pagamentos com menor Score sejam aprovados e os
maiores sejam rejeitados.

    -  Além da matriz de confusão, uma matriz associada as vendas dado as predições feitas pelo modelo
    que resultaram ou não em fraude.

        - Para a matriz de **confusão** foram selecionadas as métricas:

        \begin{align*}
        Recall =​ \frac{​TP​​}{TP+FN}​​, && && FPR = \frac{FP}{TP+FP} \\
        \end{align*}

        - Para a matriz de **custo** foi criada a métrica:

            Custo     | PRED 0         | PRED 1
            --------- | ------         | ------
            Real 0    | gain           | lost by bloq
            Real 1    | lost by fraud  | Fraud (No lost)
           

        \begin{align*}
        &Ganho\_sem\_bloq =​ 0.1*gain-lost\_by\_fraud \\
        &Ganho\_com\_bloq =​ 0.1*gain-(lost\_by\_fraud+0.1*lost\_by\_bloq) \\
        \\
        &gain:= \text{ Ganhos por acertos do modelo nas campras que não foram fraude. (TN ticket) } \\
        &lost\_by\_fraud:= \text{ Perdas por o modelo não acertar as compras que foram fraude. (FN ticket)} \\
        &lost\_by\_bloq:=\text{ Perdas por ter bloqueado a compra dado que era uma compra legítima. (FP ticket)}
        \end{align*}
        
        O ponto de corte que maximiza os ganhos do MELI é definido com aquele ponto onde as métricas de "Ganho" tenham um máximo absoluto.

        Para a métrica de $Ganho\_com\_bloq$, que inclue no cálculo da perda de oportunidade daqueles pedidos que eram legítimos mas que foram bloqueados, o ponto de corte encontra-se em 86.

        Para a métrica de $Ganho\_com\_bloq$, que não inclue no cálculo da perda de oportunidade daqueles pedidos que eram legítimos mas que foram bloqueados, o ponto de corte encontra-se em 76.



In [56]:
@interact

def slice_score_in_matrix_confusion(score = (0,100), option = [False, True]):


    confusion_matrix, confusion_matrix_monto = metrics_evaluation.cross_tab_matrices(df_origin, cutoff= score, total = option)

    ######################### Métricas ##################################################
    TP, FP, TN, FN, P, N = metrics_evaluation.confusion_matrix_2_TP_FP_TN_FN( confusion_matrix, total = option  )
    gain, lost_by_bloq, lost_by_fraud, success_by_fraud, total_sell = metrics_evaluation.confusion_matrix_2_bussines(confusion_matrix_monto, total = option)

    Recall = TP/P
    FPR = FP/(FP+TP)


    ganho_com_bloq = (0.1*gain-lost_by_fraud-0.1*lost_by_bloq)
    ganho_sem_bloq = (0.1*gain-lost_by_fraud)


    print("\n\n","========================== Confusion Matrix =======================================")
    display( confusion_matrix )


    print("Total fraudes: ", P)
    print("Total Não fraudes: ", N)
    print("\n\n")
    print("Recall: ", Recall)
    print("FPR: ", FPR)


    print("\n\n","========================== Ticket confusion Matrix =======================================")
    display( confusion_matrix_monto  )

    print("Total vendido: ", total_sell)
    print("\n\n")
    print("ganho_com_bloq: ", ganho_com_bloq)
    print("ganho_sem_bloq: ", ganho_sem_bloq)


interactive(children=(IntSlider(value=50, description='score'), Dropdown(description='option', options=(False,…

# PLOT

In [31]:
dic_gain = {"score":[], "TP":[], "FP":[], "TN":[], "FN":[],"P":[], "N":[],
             "gain":[],"lost_by_bloq":[], "lost_by_fraud":[], "success_by_fraud":[],"total_sell":[],
             "Recall":[], "FPR":[],
             "ganho_com_bloq":[], "ganho_sem_bloq":[]}

for score in range(0,101):

    TP, FP, TN, FN, P, N,\
    gain, lost_by_bloq, lost_by_fraud, success_by_fraud, total_sell,\
    Recall, FPR, ganho_com_bloq, ganho_sem_bloq = metrics_evaluation.gain_value( df = df_origin, score = score )

    dic_gain["score"].append(score)

    dic_gain["TP"].append(TP)
    dic_gain["FP"].append(FP)
    dic_gain["TN"].append(TN)
    dic_gain["FN"].append(FN)
    dic_gain["P"].append(P)
    dic_gain["N"].append(N)

    dic_gain["gain"].append(gain)
    dic_gain["lost_by_bloq"].append(lost_by_bloq)
    dic_gain["lost_by_fraud"].append(lost_by_fraud)
    dic_gain["success_by_fraud"].append(success_by_fraud)
    dic_gain["total_sell"].append(total_sell)

    ## Métricas de modelo
    dic_gain["Recall"].append( Recall )
    dic_gain["FPR"].append(FPR)

    ## Métricas de negócio
    dic_gain["ganho_com_bloq"].append(ganho_com_bloq)
    dic_gain["ganho_sem_bloq"].append(ganho_sem_bloq)


In [47]:
!dir .\..\\data

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\data

04/12/2022  11:04    <DIR>          .
23/11/2022  00:25    <DIR>          ..
04/12/2022  11:04            24.658 cat_dataset.csv
10/08/2021  12:08        26.677.890 dados.csv
28/11/2022  00:10        14.104.434 dataset_structed.parquet
28/11/2022  00:05        27.547.133 EA.pbix
22/11/2022  22:34            39.018 iv_woe_by_week.parquet
09/11/2022  10:35            54.091 MySQL_localhost_connection.pbix
05/12/2022  19:33         3.891.056 test_dataset.parquet
05/12/2022  19:33         8.711.592 train_dataset.parquet
05/12/2022  19:33        14.493.356 train_test_dataset_tranformed.parquet
05/12/2022  19:33         2.094.822 validation_dataset.parquet
              10 arquivo(s)     97.638.050 bytes
               2 pasta(s)   361.356.763.136 bytes dispon�veis


'c:\\Users\\caanp\\OneDrive\\Documents\\repositories\\ML-anti-fraud\\data\\gain.parquet'

In [55]:
df_gain = pd.DataFrame(dic_gain)

path_save = os.path.join(parameters.data_path,"cutoff.parquet")
df_gain.to_parquet(path_save, index=False)
# df_gain

In [41]:
import plotly.express as px

fig = px.line(df_gain, x="score", y=["ganho_com_bloq","ganho_sem_bloq"], title='ganho vs score')

fig.update_layout(
    autosize=False,
    width=600,
    height=600,)

fig.show()

# Requirements

In [110]:
path_root = parameters.root
!pip list --format=freeze > $path_root/requirements.txt

5697.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
